# Multi-Agent System Tutorial
In this tutorial, we will walk through the process of setting up a multi-agent system using the `crewai`, `crewai_tools`, and `langchain` libraries. The agents we will create will have specific roles and goals, such as researching quantum computing and writing engaging keynote speeches.

## Step 1: Install Necessary Libraries


Before we begin, make sure you have the necessary libraries installed. If you're using a specific package like crewai, ensure that it's available, or replace it with relevant ones as needed. Install the required libraries by running:

In [ ]:
!pip install crewai langchain langchain-community langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 808.0 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of clarifai to determ

## Step 2: Set Up the API Key

In this step, we’ll save the necessary API key required to access the language models.

In [ ]:
import os

# Save the API key
os.environ["GROQ_API_KEY"] = "your_api_key"

Replace `"api_key"` with your actual API key. This key will be used to authenticate requests to the LLM models in the subsequent steps.

## Step 3: Create the LLM for Agents

Now, we will create a language model using the `ChatGroq` class. This model will be used by both the email writer and the shadow editor. We’ll configure the model with specific parameters such as temperature, timeout, and retries.

In [ ]:
from langchain_groq import ChatGroq

# Create the LLM for writing and editing
llm = ChatGroq(
    model="groq/llama-3.1-70b-versatile",
    temperature=0.7,  # Adjusts creativity
    max_tokens=None,
    timeout=None,
    max_retries=2
)


In this step:

* **`temperature=0.7`** ensures that the LLM generates slightly more creative, varied responses. You can adjust this parameter based on the level of creativity you need.
* **`max_retries=2`** sets how many times the model will retry in case of a failure.

## Step 4: Create the Email Writer Agent

Next, we’ll create the email writer agent using the `Agent` class. This agent will take the role of writing an initial email draft.

In [ ]:
from crewai import Agent

# Create the primary agent (email writer)
email_writer = Agent(
    llm=llm,
    role="Professional Email Writer",
    goal="Write a clear and concise email to a potential client.",
    backstory="You are an experienced writer with a background in professional communication.",
    allow_delegation=False,
    verbose=1,  # Enables detailed logging
)

Explanation:

* The agent takes the role of a **"Professional Email Writer"**, and its goal is to write an email to a potential client.
* `backstory`: This provides context for the agent. It assumes the agent has experience in professional communication, which can subtly guide the LLM’s behavior.
* `allow_delegation=False`: This ensures that the agent won’t delegate the task to other agents.

## Step 5: Define the Task for Writing the Email

We now need to define a task for the email writer agent. This task will ask the agent to write an email introducing a new product to a potential client.

In [ ]:
from crewai import Task

# Task 1: Writing the initial email
task1 = Task(
    description="Write a professional email to introduce a new product to a potential client.",
    expected_output="A clear and concise email with a subject line, greeting, body, and signature.",
    output_file="email_draft.txt",
    agent=email_writer,
)

Explanation:

* The task description defines what the agent is supposed to do (write a professional email).
* The expected output specifies the format: a well-structured email with a subject, greeting, body, and signature.
* The agent’s output will be saved to `email_draft.txt`.

## Step 6: Create the Shadow Editor Agent

Once the email draft is written, we need a second agent, the shadow editor, to review and improve the email. This agent will act as a second pair of eyes, refining the tone, clarity, and professionalism.

In [ ]:
# Create the shadow editor agent
shadow_editor = Agent(
    llm=llm,
    role="Shadow Editor",
    goal="Edit the email draft for clarity, tone, and professionalism.",
    backstory="You are a seasoned editor with expertise in business communication.",
    allow_delegation=False,
    verbose=1,
)

Explanation:

* This agent's role is to act as a shadow editor, focusing on refining the original email draft.
* Its backstory emphasizes expertise in business communication, guiding it to provide high-quality edits.

## Step 7: Define the Task for Refining the Email

Now, we will create a task for the shadow editor. This task will ask the editor to take the email draft from the previous step and improve it.

In [ ]:
# Task 2: Refining the email
task2 = Task(
    description="Review and edit the email draft for clarity, tone, and professionalism.",
    expected_output="A polished version of the email that improves upon clarity and professionalism.",
    output_file="final_email.txt",
    agent=shadow_editor,
    input_file="email_draft.txt",  # Use the output of the first task as input
)

Explanation:

* The task’s description clearly states that the editor will review the draft for improvements.
* The output will be saved as `final_email.txt`.
* The `input_file="email_draft.txt"` argument ensures that the editor uses the email draft produced by the email writer.

**Note:** You can also remove the `input_file` and the code will run correctly.

## Step 8: Combine Agents and Tasks into a Crew

Finally, we will combine both agents and their respective tasks using the Crew class. This class manages the workflow by ensuring that each agent completes its task in sequence.

In [ ]:
from crewai import Crew

# Combine agents and tasks with the crew
crew = Crew(agents=[email_writer, shadow_editor], tasks=[task1, task2], verbose=1)

# Start the task execution
print(crew.kickoff())

Explanation:

* The **crew** is made up of both the email writer and the shadow editor, each performing their respective tasks.
* The `kickoff()` method will initiate the process. The email writer will first generate the email, followed by the shadow editor who will refine it.

## Execute the Workflow

Once you run the full script, the process will begin as follows:

1. The email writer agent will create the first draft and save it as `email_draft.txt`.
2. The shadow editor agent will take this draft, refine it, and save the final polished version as `final_email.txt`.

By the end, you will have a refined, professional email ready for use.